# Scraping FBREF

In [ ]:
import pandas as pd
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import locale
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import ssl
import scipy as scipy
from random import randint
from time import sleep
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
import patsy

# Web Scrapping du site fbref.com
Pour obtenir notre base de données, nous allons scrapper différentes statistiques liés à la dernière saison (2023-2024) et provenant du site fbref.com

Elle sera composé de critères provenant de tableaux différents :
- Les statistiques générales (standard stats)
- Les statistiques de tirs (shooting)
- Les statistiques de passes (pass)
- Les statistiques de défense (def)
- Les statistiques de création de buts et de tirs (gca)
- Les statistiques divers (misc)

# Statistiques générales (standard stats)

In [ ]:
url_stats = 'https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats'


df_stats = pd.read_html(url_stats)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_stats = df_stats.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_stats.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_stats.columns = new_columns


# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_stats['Age'] = df_stats['Age'].str[:2]


# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_stats['Pos'] = df_stats['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_stats['Nation'] = df_stats['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_stats['Comp'] = df_stats['Comp'].apply(remove_elements)

df_stats = df_stats.drop(columns=['Rk','Matches'])

df_stats = df_stats.dropna()

df_stats.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Playing Time Starts,Playing Time Min,...,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes G+A,Per 90 Minutes G-PK,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,3,1,86,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,4,1,120,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,1,1,65,...,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.10,0.10,0.10
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,6,0,88,...,0.00,0.00,0.00,0.00,0.00,0.06,0.00,0.06,0.06,0.06
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,1,0,3,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Statistiques de tirs (shooting)

In [ ]:
url_shot = 'https://fbref.com/en/comps/Big5/shooting/players/Big-5-European-Leagues-Stats'

df_shot = pd.read_html(url_shot)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_shot.columns = [' '.join(col).strip() for col in df_shot.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_shot = df_shot.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_shot.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_shot.columns = new_columns


# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_shot['Age'] = df_shot['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_shot['Pos'] = df_shot['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_shot['Nation'] = df_shot['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_shot['Comp'] = df_shot['Comp'].apply(remove_elements)
df_shot = df_shot.drop(columns=['Rk','Matches'])

df_shot=df_shot.fillna(0)

# Supprimer les lignes contenant le mot "Player" dans n'importe quelle colonne
df_shot = df_shot[~df_shot.apply(lambda row: row.astype(str).str.contains("Player", case=False, na=False).any(), axis=1)]

df_shot = df_shot[~df_shot[['Nation', 'Pos','Age', 'Born']].isin([0]).any(axis=1)]


df_shot.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Standard Gls,Standard Sh,...,Standard G/SoT,Standard Dist,Standard FK,Standard PK,Standard PKatt,Expected xG,Expected npxG,Expected npxG/Sh,Expected G-xG,Expected np:G-xG
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,1.0,0,0,...,0,0,0,0,0,0.0,0.0,0,0.0,0.0
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,1.3,0,0,...,0,0,0,0,0,0.0,0.0,0,0.0,0.0
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,0.7,0,1,...,0,24.5,0,0,0,0.1,0.1,0.07,-0.1,-0.1
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,1.0,0,1,...,0,15.0,0,0,0,0.1,0.1,0.06,-0.1,-0.1
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0.0,0,0,...,0,0,0,0,0,0.0,0.0,0,0.0,0.0


# Statistiques de passes (pass)

In [ ]:
# URL de la page contenant les statistiques de passes pour les joueurs des cinq grands championnats européens
pass_url = 'https://fbref.com/en/comps/Big5/passing/players/Big-5-European-Leagues-Stats'

# Charger les données du tableau HTML en un DataFrame
df_pass = pd.read_html(pass_url)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_pass.columns = [' '.join(col).strip() for col in df_pass.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_pass = df_pass.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_pass.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_pass.columns = new_columns

# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_pass['Age'] = df_pass['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_pass['Pos'] = df_pass['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_pass['Nation'] = df_pass['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_pass['Comp'] = df_pass['Comp'].apply(remove_elements)
df_pass = df_pass.drop(columns=['Rk','Matches'])

df_pass=df_pass.fillna(0)

df_pass = df_pass[~df_pass.apply(lambda row: row.astype(str).str.contains("Player", case=False, na=False).any(), axis=1)]

df_pass = df_pass[~df_pass[['Nation', 'Pos','Age', 'Born']].isin([0]).any(axis=1)]


df_pass.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Total Cmp,Total Att,...,Long Cmp%,Ast,xAG,Expected xA,Expected A-xAG,KP,1/3,PPA,CrsPA,PrgP
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,1.0,50,63,...,57.1,0,0.0,0.0,0.0,0,8,0,0,8
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,1.3,47,66,...,25.0,0,0.0,0.0,0.0,1,2,0,0,6
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,0.7,17,29,...,28.6,0,0.0,0.0,0.0,0,0,0,0,2
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,1.0,36,46,...,37.5,0,0.0,0.0,0.0,0,2,0,0,4
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0.0,2,2,...,0,0,0.0,0.0,0.0,0,0,0,0,0


# Statistiques de défense (def)

In [ ]:
# URL de la page contenant les statistiques de passes pour les joueurs des cinq grands championnats européens
def_url = 'https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats'

# Charger les données du tableau HTML en un DataFrame
df_def = pd.read_html(def_url)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_def.columns = [' '.join(col).strip() for col in df_def.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_def = df_def.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_def.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_def.columns = new_columns

# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_def['Age'] = df_def['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_def['Pos'] = df_def['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_def['Nation'] = df_def['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_def['Comp'] = df_def['Comp'].apply(remove_elements)
df_def = df_def.drop(columns=['Rk','Matches'])

df_def=df_def.fillna(0)

df_def = df_def[~df_def.apply(lambda row: row.astype(str).str.contains("Player", case=False, na=False).any(), axis=1)]

# Supprimer les lignes où "Nation", "Age" ou "Born" contiennent 0
df_def = df_def[~df_def[['Nation', 'Pos','Age', 'Born']].isin([0]).any(axis=1)]

df_def.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tackles Tkl,Tackles TklW,...,Challenges Att,Challenges Tkl%,Challenges Lost,Blocks Blocks,Blocks Sh,Blocks Pass,Int,Tkl+Int,Clr,Err
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,1.0,2,2,...,1,100.0,0,3,1,2,1,3,0,0
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,1.3,4,4,...,5,80.0,1,1,0,1,0,4,3,0
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,0.7,3,2,...,3,100.0,0,1,0,1,1,4,0,0
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,1.0,4,2,...,3,66.7,1,2,1,1,1,5,3,0
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0.0,1,1,...,1,100.0,0,0,0,0,0,1,0,0


# Statistiques de création de buts et de tirs (gca)

In [ ]:
# URL de la page contenant les statistiques des passes décisives créant des occasions des joueurs des cinq grands championnats européens
gca_url = 'https://fbref.com/en/comps/Big5/gca/players/Big-5-European-Leagues-Stats'

# Charger les données du tableau HTML en un DataFrame
df_gca = pd.read_html(gca_url)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_gca.columns = [' '.join(col).strip() for col in df_gca.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_gca = df_gca.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_gca.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_gca.columns = new_columns

# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_gca['Age'] = df_gca['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos'
df_gca['Pos'] = df_gca['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_gca['Nation'] = df_gca['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_gca['Comp'] = df_gca['Comp'].apply(remove_elements)
df_gca = df_gca.drop(columns=['Rk','Matches'])

df_gca = df_gca.dropna()

df_gca.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,SCA SCA,SCA SCA90,...,SCA Types Fld,SCA Types Def,GCA GCA,GCA GCA90,GCA Types PassLive,GCA Types PassDead,GCA Types TO,GCA Types Sh,GCA Types Fld,GCA Types Def
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,1.0,2,2.09,...,0,0,0,0.00,0,0,0,0,0,0
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,1.3,1,0.75,...,0,0,0,0.00,0,0,0,0,0,0
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,0.7,0,0.00,...,0,0,0,0.00,0,0,0,0,0,0
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,1.0,1,1.02,...,0,0,0,0.00,0,0,0,0,0,0
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0.0,0,0.00,...,0,0,0,0.00,0,0,0,0,0,0


# Statistiques divers (misc)

In [ ]:
# URL de la page contenant les statistiques diverses des joueurs des cinq grands championnats européens
misc_url = 'https://fbref.com/en/comps/Big5/misc/players/Big-5-European-Leagues-Stats'

# Charger les données du tableau HTML en un DataFrame
df_misc = pd.read_html(misc_url)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_misc.columns = [' '.join(col).strip() for col in df_misc.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_misc = df_misc.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_misc.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_misc.columns = new_columns

# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_misc['Age'] = df_misc['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_misc['Pos'] = df_misc['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_misc['Nation'] = df_misc['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_misc['Comp'] = df_misc['Comp'].apply(remove_elements)
df_misc = df_misc.drop(columns=['Rk','Matches'])

df_misc=df_misc.fillna(0)

df_misc = df_misc[~df_misc.apply(lambda row: row.astype(str).str.contains("Player", case=False, na=False).any(), axis=1)]

df_misc = df_misc[~df_misc[['Nation', 'Pos','Age', 'Born']].isin([0]).any(axis=1)]


df_misc.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Performance CrdY,Performance CrdR,...,Performance Crs,Performance Int,Performance TklW,Performance PKwon,Performance PKcon,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,1.0,0,0,...,2,1,2,0,0,0,7,0,0,0
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,1.3,2,0,...,5,0,4,0,0,0,7,2,1,66.7
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,0.7,1,0,...,1,1,2,0,0,0,2,0,1,0.0
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,1.0,1,0,...,0,1,2,0,0,0,7,2,2,50.0
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0.0,0,0,...,0,0,1,0,0,0,0,0,0,0


# Statistiques de possession (poss)

In [ ]:
# Définition de l'URL contenant les données sur la possession des joueurs
poss_url  = 'https://fbref.com/en/comps/Big5/possession/players/Big-5-European-Leagues-Stats'

# Lecture des tables HTML depuis l'URL et stockage de la première table dans un DataFrame
df_poss = pd.read_html(poss_url)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_poss.columns = [' '.join(col).strip() for col in df_poss.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_poss = df_poss.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_poss.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_poss.columns = new_columns

# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_poss['Age'] = df_poss['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_poss['Pos'] = df_poss['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_poss['Nation'] = df_poss['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_poss['Comp'] = df_poss['Comp'].apply(remove_elements)
df_poss = df_poss.drop(columns=['Rk','Matches'])

df_poss=df_poss.fillna(0)
df_poss = df_poss[~df_poss.apply(lambda row: row.astype(str).str.contains("Player", case=False, na=False).any(), axis=1)]

df_poss = df_poss[~df_poss[['Nation', 'Pos','Age', 'Born']].isin([0]).any(axis=1)]


df_poss.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Touches Touches,Touches Def Pen,...,Carries Carries,Carries TotDist,Carries PrgDist,Carries PrgC,Carries 1/3,Carries CPA,Carries Mis,Carries Dis,Receiving Rec,Receiving PrgR
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,1.0,73,2,...,41,152,68,1,0,0,1,0,40,3
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,1.3,85,1,...,46,215,103,0,3,0,3,1,49,10
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,0.7,36,1,...,13,101,67,3,2,1,0,2,16,3
3,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,1.0,65,8,...,29,219,165,3,1,1,1,3,34,1
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0.0,4,0,...,1,10,9,1,0,0,1,0,3,0


# Statistiques de temps de jeu (playTime)

In [ ]:
# Définition de l'URL contenant les données sur le temps de jeu des joueurs
play_url = 'https://fbref.com/en/comps/Big5/playingtime/players/Big-5-European-Leagues-Stats'

# Lecture des tables HTML depuis l'URL et stockage de la première table dans un DataFrame
df_play = pd.read_html(play_url)[0]

# Supprimer le multi-indexing en combinant les niveaux de colonnes en une seule ligne de noms
df_play.columns = [' '.join(col).strip() for col in df_play.columns]

# Réinitialiser l'index pour que chaque ligne soit indexée de 0 à n-1
df_play = df_play.reset_index(drop=True)

# Créer une liste pour stocker les nouveaux noms de colonnes
new_columns = []
for col in df_play.columns:
    if 'level_0' in col:
        # Si le nom de la colonne contient 'level_0', prendre le dernier mot comme nouveau nom de colonne
        new_col = col.split()[-1]
    else:
        # Sinon, garder le nom de colonne tel quel
        new_col = col
    # Ajouter le nom de colonne à la liste new_columns
    new_columns.append(new_col)

# Renommer les colonnes en utilisant la liste new_columns
df_play.columns = new_columns

# Garder seulement les deux premiers caractères de la colonne 'Age' (pour l'âge du joueur)
df_play['Age'] = df_play['Age'].str[:2]

# Extraction des deux premiers caractères de la colonne 'Pos' pour créer une nouvelle colonne 'Position'
df_play['Pos'] = df_play['Pos'].str[:2]


# Extraire le code de la nation à partir du deuxième mot dans la colonne 'Nation'
df_play['Nation'] = df_play['Nation'].str.split(' ').str.get(1)

def remove_elements(text):
  elements_to_remove = ["eng", "es", "it", "fr", "de"]
  for element in elements_to_remove:
    text = text.replace(element, "").strip()
  return text

df_play['Comp'] = df_play['Comp'].apply(remove_elements)
df_play = df_play.drop(columns=['Rk','Matches'])

df_play=df_play.fillna(0)

df_play = df_play[~df_play.apply(lambda row: row.astype(str).str.contains("Player", case=False, na=False).any(), axis=1)]

df_play = df_play[~df_play[['Nation', 'Pos','Age', 'Born']].isin([0]).any(axis=1)]


df_play.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Playing Time Min,Playing Time Mn/MP,...,Team Success onG,Team Success onGA,Team Success +/-,Team Success +/-90,Team Success On-Off,Team Success (xG) onxG,Team Success (xG) onxGA,Team Success (xG) xG+/-,Team Success (xG) xG+/-90,Team Success (xG) On-Off
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,3,86,29,...,2,0,+2,+2.09,+1.82,2.3,0.3,+2.0,+2.12,+1.75
1,Max Aarons,ENG,DF,Valencia,La Liga,24,2000,4,120,30,...,1,3,-2,-1.50,-1.28,1.5,3.7,-2.3,-1.69,-1.52
2,Álvaro Abajas,ESP,GK,Leganés,La Liga,21,2003,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21,2003,1,65,65,...,1,2,-1,-1.38,-1.14,1.4,0.7,+0.7,+0.93,+1.18
4,James Abankwah,IRL,DF,Udinese,Serie A,20,2004,6,88,15,...,2,0,+2,+2.05,+2.50,0.6,1.5,-0.9,-0.91,-0.54


In [ ]:
# Vérifiez les doublons dans les colonnes de jointure
print(df_stats.duplicated(subset=['Player', 'Squad']).sum())
print(df_gca.duplicated(subset=['Player', 'Squad']).sum())
print(df_pass.duplicated(subset=['Player', 'Squad']).sum())
print(df_misc.duplicated(subset=['Player', 'Squad']).sum())
print(df_shot.duplicated(subset=['Player', 'Squad']).sum())
print(df_def.duplicated(subset=['Player', 'Squad']).sum())
print(df_play.duplicated(subset=['Player', 'Squad']).sum())
print(df_poss.duplicated(subset=['Player', 'Squad']).sum())

0
0
0
0
0
0
1
0


In [ ]:
df_play.drop_duplicates(subset=['Player', 'Squad'], inplace=True)

Rassembler tous les datasets ensemble

In [ ]:
df = df_stats.merge(df_gca, how='inner', on=['Player', 'Squad'], suffixes=('_stats', '_gca'))
df = df.merge(df_pass, how='inner', on=['Player', 'Squad'], suffixes=('_merge1', '_pass'))
df = df.merge(df_misc, how='inner', on=['Player', 'Squad'], suffixes=('_merge2', '_misc'))
df = df.merge(df_shot, how='inner', on=['Player', 'Squad'], suffixes=('_merge3', '_shot'))
df = df.merge(df_def, how='inner', on=['Player', 'Squad'], suffixes=('_merge4', '_def'))
df = df.merge(df_play, how='inner', on=['Player', 'Squad'], suffixes=('_merge5', '_play'))
df = df.merge(df_poss, how='inner', on=['Player', 'Squad'], suffixes=('_merge6', '_poss'))

# Afficher le DataFrame
print(df.head())

              Player Nation_stats Pos_stats        Squad      Comp_stats  \
0         Max Aarons          ENG        DF  Bournemouth  Premier League   
1         Max Aarons          ENG        DF     Valencia         La Liga   
2     Rodrigo Abajas          ESP        DF     Valencia         La Liga   
3     James Abankwah          IRL        DF      Udinese         Serie A   
4  Keyliane Abdallah          FRA        FW    Marseille         Ligue 1   

  Age_stats Born_stats Playing Time MP_merge5 Playing Time Starts  \
0        24       2000                      3                   1   
1        24       2000                      4                   1   
2        21       2003                      1                   1   
3        20       2004                      6                   0   
4        18       2006                      1                   0   

  Playing Time Min_merge5  ... Carries Carries Carries TotDist  \
0                      86  ...              41             152

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2844 entries, 0 to 2843
Columns: 209 entries, Player to Receiving PrgR
dtypes: object(209)
memory usage: 4.5+ MB


In [ ]:
print(df.columns.tolist())

['Player', 'Nation_stats', 'Pos_stats', 'Squad', 'Comp_stats', 'Age_stats', 'Born_stats', 'Playing Time MP_merge5', 'Playing Time Starts', 'Playing Time Min_merge5', 'Playing Time 90s_merge5', 'Performance Gls', 'Performance Ast', 'Performance G+A', 'Performance G-PK', 'Performance PK', 'Performance PKatt', 'Performance CrdY_merge2', 'Performance CrdR_merge2', 'Expected xG_merge3', 'Expected npxG_merge3', 'Expected xAG', 'Expected npxG+xAG', 'Progression PrgC', 'Progression PrgP', 'Progression PrgR', 'Per 90 Minutes Gls', 'Per 90 Minutes Ast', 'Per 90 Minutes G+A', 'Per 90 Minutes G-PK', 'Per 90 Minutes G+A-PK', 'Per 90 Minutes xG', 'Per 90 Minutes xAG', 'Per 90 Minutes xG+xAG', 'Per 90 Minutes npxG', 'Per 90 Minutes npxG+xAG', 'Nation_gca', 'Pos_gca', 'Comp_gca', 'Age_gca', 'Born_gca', '90s_merge1', 'SCA SCA', 'SCA SCA90', 'SCA Types PassLive', 'SCA Types PassDead', 'SCA Types TO', 'SCA Types Sh', 'SCA Types Fld', 'SCA Types Def', 'GCA GCA', 'GCA GCA90', 'GCA Types PassLive', 'GCA Typ

In [ ]:
df.isna().sum()

,0
Player,0
Nation_stats,0
Pos_stats,0
Squad,0
Comp_stats,0
...,...
Carries CPA,0
Carries Mis,0
Carries Dis,0
Receiving Rec,0


In [ ]:
df.columns = df.columns.str.split('_').str[0]  # Supprime le suffixe après '_'
df = df.loc[:, ~df.columns.duplicated()]  # Supprime les doublons de colonnes

In [ ]:
# Dictionnaire de renommage des colonnes
rename_columns = {
"Player": "Joueur",
"Nation": "Nationalité",
'Pos': 'Position',
"Squad": "Équipe",
"Comp": "Compétition",
"Age": "Âge",
"Born": "Année de naissance",
"Playing Time MP": "Matchs joués",
"Playing Time Starts": "Titularisations",
"Playing Time Min": "Minutes jouées",
"Playing Time 90s": "Matchs en 90 min",
"Performance Gls": "Buts",
"Performance Ast": "Passes décisives",
"Performance G+A": "Buts + Passes D",
"Performance G-PK": "Buts (sans penalty)",
"Performance PK": "Pénaltys marqués",
"Performance PKatt": "Pénaltys tentés",
"Performance CrdY": "Cartons jaunes",
"Performance CrdR": "Cartons rouges",
"Expected xG": "Buts attendus (xG)",
"Expected npxG": "Buts attendus sans penalty",
"Expected xAG": "Passes décisives attendues (xAG)",
"Expected npxG+xAG": "xG + xAG sans penalty",
"Progression PrgC": "Courses progressives",
"Progression PrgP": "Passes progressives",
"Progression PrgR": "Réceptions progressives",
"Per 90 Minutes Gls": "Buts par 90 minutes",
"Per 90 Minutes Ast": "Passes décisives par 90 minutes",
"Per 90 Minutes G+A" : "Buts + Passes décisives par 90 minutes",
"Per 90 Minutes G-PK": "Buts hors penalty par 90 minutes",
"Per 90 Minutes G+A-PK":"Buts + Passes décisives hors penalty par 90 minutes",
"Per 90 Minutes xG": "Buts attendus par 90 minutes",
"Per 90 Minutes xAG": "Passes décisives attendues par 90 minutes",
"Per 90 Minutes xG+xAG": "Somme des buts et passes attendues par 90 minutes",
"Per 90 Minutes npxG" : "Buts attendus hors penalty par 90 minutes",
"Per 90 Minutes npxG+xAG" : "Somme des buts et passes attendues hors penalty par 90 minutes",
"90s" : "Équivalents 90 minutes joués",
"SCA SCA": "Actions menant à un tir",
"SCA SCA90": "Actions menant à un tir par 90 minutes",
"SCA Types PassLive": "Passes en jeu menant à un tir",
"SCA Types PassDead": "Passes arrêtées (coups francs, corners) menant à un tir",
"SCA Types TO": "Dribbles réussis menant à un tir",
"SCA Types Sh": "Tirs ayant provoqué un autre tir",
"SCA Types Fld": "Fautes subies menant à un tir",
"SCA Types Def": "Actions défensives menant à un tir",
"GCA GCA": "Actions menant à un but",
"GCA GCA90": "Actions menant à un but par 90 minutes",
"GCA Types PassLive" : "Passes en jeu menant à un but",
"GCA Types PassDead": "Passes arrêtées (coups francs, corners) menant à un but",
"GCA Types TO" : "Dribbles réussis menant à un but",
"GCA Types Sh" : "Tirs ayant provoqué un but",
"GCA Types Fld": "Fautes subies menant à un but",
"GCA Types Def": "Actions défensives menant à un but",
"Total Cmp": "Passes réussies",
"Total Att": "Passes tentées",
"Total Cmp%" : "Pourcentage de passes réussies",
"Total TotDist" : "Distance totale des passes",
"Total PrgDist" : "Distance progressive des passes",
"Short Cmp" : "Passes courtes réussies",
"Short Att" : "Passes courtes tentées",
"Short Cmp%" : "Pourcentage de passes courtes réussies",
"Medium Cmp" : "Passes moyennes réussies",
"Medium Att" : "Passes moyennes tentées",
"Medium Cmp%" : "Pourcentage de passes moyennes réussies",
"Long Cmp" : "Passes longues réussies",
"Long Att" : "Passes longues tentées",
"Long Cmp%" : "Pourcentage de passes longues réussies",
"Ast" : "Passes décisives",
"xAG" : "Passes décisives attendues",
"Expected xA" : "Passes attendues (xA)",
"Expected A-xAG" : "Différence entre passes décisives réelles et attendues",
"KP" : "Passes clés",
"1/3" : "Passes dans le dernier tiers",
"PPA" : "Passes dans la surface",
"CrsPA" : "Centres dans la surface",
"PrgP" : "Passes progressives",
"Performance 2CrdY" : "Deuxième carton jaune",
"Performance Fls" : "Fautes commises",
"Performance Fld" : "Fautes subies",
"Performance Off" : "Hors-jeux",
"Performance Crs" : "Centres tentés",
"Performance Int" : "Interceptions",
"Performance TklW" : "Tacles gagnants",
"Performance PKwon" : "Penaltys provoqués",
"Performance PKcon" : "Penaltys concédés",
"Performance OG" : "Buts contre son camp",
"Performance Recov" : "Ballons récupérés",
"Aerial Duels Won" : "Duels aériens gagnés",
"Aerial Duels Lost" : "Duels aériens perdus",
"Aerial Duels Won%" : "Pourcentage de duels aériens gagnés",
"Standard Gls": "Buts",
"Standard Sh" : "Tirs",
"Standard SoT" : "Tirs cadrés",
"Standard SoT%" : "Pourcentage de tirs cadrés",
"Standard Sh/90" : "Tirs par 90 minutes",
"Standard SoT/90" : "Tirs cadrés par 90 minutes",
"Standard G/Sh" : "Buts par tir",
"Standard G/SoT" : "Buts par tir cadré",
"Standard Dist" : "Distance moyenne des tirs",
"Standard FK" : "Coups francs tentés",
"Standard PK" : "Penaltys marqués",
"Standard PKatt" : "Penaltys tentés",
"Expected npxG/Sh" : "Buts attendus hors penalty par tir",
"Expected G-xG" : "Différence entre les buts marqués et les buts attendus",
"Expected np:G-xG" : "Différence entre les buts marqués hors penalty et les buts attendus hors penalty",
"Tackles Tkl" : "Tacles réussis",
"Tackles TklW" : "Tacles gagnants",
"Tackles Def 3rd" : "Tacles réussis dans le tiers défensif",
"Tackles Mid 3rd" : "Tacles réussis dans le tiers médian",
"Tackles Att 3rd" : "Tacles réussis dans le tiers offensif",
"Challenges Tkl" : "Duels défensifs gagnés",
"Challenges Att" : "Duels défensifs disputés",
"Challenges Tkl%" : "Pourcentage de duels gagnés",
"Challenges Lost" : "Duels défensifs perdus",
"Blocks Blocks" : "Total de blocs (tirs et passes)",
"Blocks Sh" : "Tirs bloqués",
"Blocks Pass" : "Passes bloquées",
"Int" : "Interceptions",
"Tkl+Int" : "Total de tacles et d’interceptions",
"Clr" : "Dégagements",
"Err" : "Erreurs menant à un tir ou un but",
"Playing Time Mn/MP" : "Minutes jouées par match",
"Playing Time Min%" : "Pourcentage de minutes jouées",
"Starts Starts" : "Matches débutés en tant que titulaire",
"Starts Mn/Start" : "Minutes jouées par titularisation",
"Starts Compl" : "Matches joués en intégralité",
"Subs Subs" : "Nombre d’entrées en jeu",
"Subs Mn/Sub" : "Minutes jouées par entrée en jeu",
"Subs unSub" : "Matches passés sur le banc sans entrer en jeu",
"Team Success PPM" : "Points par match (PPM)",
"Team Success onG" : "Buts marqués par l’équipe lorsque le joueur est sur le terrain",
"Team Success onGA" : "Buts encaissés par l’équipe lorsque le joueur est sur le terrain",
"Team Success +/-" : "Différence de buts lorsque le joueur est sur le terrain",
"Team Success +/-90" : "Différence de buts par 90 minutes",
"Team Success On-Off" : "Impact du joueur sur la différence de buts par rapport au temps passé sur le terrain",
"Team Success (xG) onxG" : "xG de l’équipe lorsque le joueur est sur le terrain",
"Team Success (xG) onxGA" : "xG concédés par l’équipe lorsque le joueur est sur le terrain",
"Team Success (xG) xG+/-" : "Différence xG lorsque le joueur est sur le terrain",
"Team Success (xG) xG+/-90" : "Différence xG par 90 minutes",
"Team Success (xG) On-Off" : "Impact du joueur sur la différence xG lorsqu’il est sur le terrain ou non",
"Touches Touches": "Touches de balle",
"Touches Def Pen" : "Touches de balle dans la surface défensive",
"Touches Def 3rd" : "Touches de balle dans le tiers défensif",
"Touches Mid 3rd" : "Touches de balle dans le tiers médian",
"Touches Att 3rd" : "Touches de balle dans le tiers offensif",
"Touches Att Pen" : "Touches de balle dans la surface offensive",
"Touches Live" : "Touches de balle en jeu (hors coups de pied arrêtés)",
"Take-Ons Att" : "Dribbles tentés",
"Take-Ons Succ" : "Dribbles réussis",
"Take-Ons Succ%" : "Pourcentage de dribbles réussis",
"Take-Ons Tkld" : "Dribbles stoppés par l’adversaire",
"Take-Ons Tkld%" : "Pourcentage de dribbles stoppés",
"Carries Carries" : "Portées de balle",
"Carries TotDist" : "Distance totale parcourue avec le ballon (en mètres)",
"Carries PrgDist" : "Distance progressive parcourue avec le ballon",
"Carries PrgC" : "Portées de balle progressives",
"Carries 1/3" : "Portées de balle jusqu’au dernier tiers du terrain",
"Carries CPA" : "Portées de balle entrant dans la surface adverse",
"Carries Mis" : "Ballons perdus en conduite",
"Carries Dis" : "Ballons perdus sous la pression d’un adversaire",
"Receiving Rec" : "Passes reçues",
"Receiving PrgR" : "Passes progressives reçues"
}

# Appliquer le renommage des colonnes
df = df.rename(columns=rename_columns)

In [ ]:
#Modifier la position de Nicolás González (ESP, Manchester City) en Milieu Central
df.loc[
    (df["Joueur"] == "Nicolás González") &
    (df["Nationalité"] == "ESP") &
    (df["Équipe"] == "Manchester City"),
    "Position"
] = "MF"

# 🔹 Renommer Joshua King en Josh King (ENG) et changer sa position en MC
df.loc[
    (df["Joueur"] == "Joshua King") &
    (df["Nationalité"] == "ENG"),
    ["Joueur", "Position"]
] = ["Josh King", "MF"]

# 🔹 Renommer Rodri en Rodri Sánchez (ESP) et changer sa position en Ailier Droit
df.loc[
    (df["Joueur"] == "Rodri") &
    (df["Équipe"] == "Betis"),
    ["Joueur", "Position"]
] = ["Rodri Sánchez", "FW"]

# Modifier la position de Vitinha (POR, FW, Genoa) en Attaquant Central et changer son nom
df.loc[
    (df["Joueur"] == "Vitinha") &
    (df["Équipe"] == "Genoa"),
    ["Joueur", "Position"] # Sélectionnez les colonnes 'Joueur' et 'Position'
] = ["Vitor Vitinha", "FW"] # Attribuez les nouvelles valeurs

In [ ]:
df.to_csv('df_BIG2025.csv', sep=',', encoding='utf-8')

In [ ]:
df = pd.read_csv("df_BIG2025.csv", sep=",")

In [ ]:
df['Compétition'] = df['Compétition'].replace('Bundliga', 'Bundesliga')

In [ ]:
# Conversion des valeurs de la colonne "Position"
df['Position'] = df['Position'].replace({
    'DF': 'Défenseur',
    'MF': 'Milieu',
    'FW': 'Attaquant',
    'GK': 'Gardien'
})

# Définir les ligues et positions
ligues = ['La Liga', 'Ligue 1', 'Premier League', 'Serie A', 'Bundesliga']
positions = ['Défenseur', 'Milieu', 'Attaquant', 'Gardien']

from google.colab import files

fichiers_a_telecharger = []

# Créer un fichier CSV pour chaque combinaison de ligue et de position
for ligue in ligues:
    for position in positions:
        # Filtrer les joueurs correspondant à la ligue et à la position
        df_filtered = df[(df['Compétition'] == ligue) & (df['Position'] == position)]

        # Si des données sont présentes pour cette combinaison, les enregistrer dans un fichier CSV
        if not df_filtered.empty:
            file_name = f"{ligue}_{position}.csv".replace(" ", "_")
            df_filtered.to_csv(file_name, index=False)
            fichiers_a_telecharger.append(file_name) # Ajouter le nom du fichier à la liste
            print(f"Fichier {file_name} créé avec succès.")

Fichier La_Liga_Défenseur.csv créé avec succès.
Fichier La_Liga_Milieu.csv créé avec succès.
Fichier La_Liga_Attaquant.csv créé avec succès.
Fichier La_Liga_Gardien.csv créé avec succès.
Fichier Ligue_1_Défenseur.csv créé avec succès.
Fichier Ligue_1_Milieu.csv créé avec succès.
Fichier Ligue_1_Attaquant.csv créé avec succès.
Fichier Ligue_1_Gardien.csv créé avec succès.
Fichier Premier_League_Défenseur.csv créé avec succès.
Fichier Premier_League_Milieu.csv créé avec succès.
Fichier Premier_League_Attaquant.csv créé avec succès.
Fichier Premier_League_Gardien.csv créé avec succès.
Fichier Serie_A_Défenseur.csv créé avec succès.
Fichier Serie_A_Milieu.csv créé avec succès.
Fichier Serie_A_Attaquant.csv créé avec succès.
Fichier Serie_A_Gardien.csv créé avec succès.
Fichier Bundesliga_Défenseur.csv créé avec succès.
Fichier Bundesliga_Milieu.csv créé avec succès.
Fichier Bundesliga_Attaquant.csv créé avec succès.
Fichier Bundesliga_Gardien.csv créé avec succès.
